# Scraping Vehicule neuf 

In [2]:
#import packages
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import io
import requests
import pandas as pd
import re

C:\Users\MSI\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# Cell 2: Initialize the Chrome driver
service = Service(r'C:\chromedriver.exe')  # Path to your chromedriver

chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-default-apps")
chrome_options.add_argument("--no-first-run")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--disable-infobars")

#Lancer le navigateur avec ChromeDriver
browser = webdriver.Chrome(service=service, options=chrome_options)
# Ouvrir le site web
browser.get('http://www.google.com')

In [5]:
# time.sleep(2)
browser.get("https://www.automobile.tn/fr/neuf")

car_response = browser.page_source
car_soup = BeautifulSoup(car_response, 'html.parser')

In [11]:
#get all the brands
pattern = re.compile(r"^/fr/neuf/.+")
brands = []
for a_tag in car_soup.find_all('a', href=True):
    href = a_tag['href']
    if pattern.match(href):
        brands.append("https://www.automobile.tn"+href)
        
brands = brands[:-3]
brands

['https://www.automobile.tn/fr/neuf/electrique',
 'https://www.automobile.tn/fr/neuf/concessionnaires',
 'https://www.automobile.tn/fr/neuf/comparateur',
 'https://www.automobile.tn/fr/neuf/comparateur',
 'https://www.automobile.tn/fr/neuf/alfa-romeo',
 'https://www.automobile.tn/fr/neuf/audi',
 'https://www.automobile.tn/fr/neuf/bako',
 'https://www.automobile.tn/fr/neuf/bestune',
 'https://www.automobile.tn/fr/neuf/bmw',
 'https://www.automobile.tn/fr/neuf/byd',
 'https://www.automobile.tn/fr/neuf/changan',
 'https://www.automobile.tn/fr/neuf/chery',
 'https://www.automobile.tn/fr/neuf/chevrolet',
 'https://www.automobile.tn/fr/neuf/citroen',
 'https://www.automobile.tn/fr/neuf/cupra',
 'https://www.automobile.tn/fr/neuf/dacia',
 'https://www.automobile.tn/fr/neuf/dfsk',
 'https://www.automobile.tn/fr/neuf/dongfeng',
 'https://www.automobile.tn/fr/neuf/faw',
 'https://www.automobile.tn/fr/neuf/fiat',
 'https://www.automobile.tn/fr/neuf/foday',
 'https://www.automobile.tn/fr/neuf/ford

In [12]:
def get_cars():
    
    car_response = browser.page_source
    car_soup = BeautifulSoup(car_response, 'html.parser')
    cars = car_soup.find_all('div', class_='versions-item')

    links_cars = []
    for car in cars:
        link = car.find('a')['href']
        links_cars.append("https://www.automobile.tn" + link)  # comment here mhamed
    return links_cars

In [13]:
# Cell 4: Function to scrape car specifications
def get_data(browser):
    car_specs = {}

    car_response = browser.page_source
    car_soup = BeautifulSoup(car_response, 'html.parser')

    urll = browser.current_url
    split_url = urll.split('neuf')[-1]
    split2 = split_url.split('/')
    car_specs['Marque'] = split2[1]
    car_specs['Modele'] = split2[2]

    for table in car_soup.find_all('table'):
        if 'Motorisation' in table.get_text():
            for row in table.find_all('tr'):
                th = row.find('th')
                td = row.find('td')
                if th and td:
                    th_text = th.get_text(strip=True)
                    td_text = td.get_text(strip=True)
                    if th_text == 'Energie':
                        car_specs['Energie'] = td_text
                    elif th_text == 'Puissance fiscale':
                        car_specs['Puissance fiscale'] = td_text

        elif 'Caractéristiques' in table.get_text():
            for row in table.find_all('tr'):
                th = row.find('th')
                td = row.find('td')
                if th and td:
                    th_text = th.get_text(strip=True)
                    td_text = td.get_text(strip=True)
                    if th_text == 'Nombre de places':
                        car_specs['Nombre de places'] = td_text
                    elif th_text == 'Nombre de portes':
                        car_specs['Nombre de portes'] = td_text

        elif 'Transmission' in table.get_text():
            for row in table.find_all('tr'):
                th = row.find('th')
                td = row.find('td')
                if th and td:
                    th_text = th.get_text(strip=True)
                    td_text = td.get_text(strip=True)
                    if th_text == 'Boîte':
                        car_specs['Boîte'] = td_text
                        
                        
    image_div = car_soup.find('div', class_='version-img')
    img_tag = image_div.find('img')
    if img_tag and img_tag['src']:
        car_specs['Image URL'] = img_tag['src']

    title = car_soup.title.string
    prix = title.split('-')[-1].strip()
    car_specs["Prix"] = prix

    return car_specs

In [14]:
def test_menu():
    car_response = browser.page_source
    car_soup = BeautifulSoup(car_response, 'html.parser')
    exists = car_soup.body.find_all(string= "comparer")
    if exists:
        print("mawjouda fazet select")
        # bech ye5ou awil lien mta3 awil karhba
        table = car_soup.find('table', class_='versions')
        links = []
    
        if table:
            for row in table.find_all('tr'):
                version_cell = row.find('td', class_='version')
                if version_cell:
                    link = version_cell.find('a')
                    if link:
                        full_link = "https://www.automobile.tn" + link['href']
                        links.append(full_link)
        # bech y7el lien lkarhba
        browser.get(links[0])
        time.sleep(2)
        # bech ye5ou les données mta3 el karhba
        car_specs = get_data(browser)
        return car_specs
    else:
        print("5arej direct")
        # ya3ni 5arjli direct el data maghir ma todkol ll version
        car_specs = get_data(browser)
        return car_specs

In [21]:
df = pd.DataFrame()

for brand in brands:
    time.sleep(1)
    browser.get(brand)
    links_cars = get_cars()
    
    for car in links_cars: #men lien el car 1 jusqu'a el car x
        # print(car)
        browser.get(car)  # D5alna ll el car i
        time.sleep(3)     # Nistanaw el page treloadi
        car_specs = test_menu()  # bech ta5delna el data (les informations) mta3 el car heki
        time.sleep(1)  # nistnaw 1 sec 
        df2 = pd.DataFrame([car_specs])  # nsajlou les informations fi dataframe
        df = pd.concat([df, df2], ignore_index=True) # nzidou el informations mta3 el car jdida m3a les cars l9dom
        print(car_specs)

        
        
df  # this is the display of the dataframe (tableau)

ida ken mawjouda comparer
{'Marque': 'bako', 'Modele': 'bee', 'Nombre de places': '3', 'Nombre de portes': '2', 'Energie': 'Electrique', 'Puissance fiscale': '1CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/06/47159.png?t=1720537057', 'Prix': '16 210 DT'}
ida ken mawjouda comparer
{'Marque': 'bako', 'Modele': 'evan', 'Nombre de places': '1', 'Nombre de portes': '3', 'Energie': 'Electrique', 'Puissance fiscale': 'N.C', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/08/47162.png?t=1723634043', 'Prix': '22 586 DT'}
moch mawjouda
{'Marque': 'byd', 'Modele': 'dolphin', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Electrique', 'Puissance fiscale': '5CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/06/47172.jpg?t=1723020945', 'Prix': '94 990 DT'}
moch mawjouda
{'Marque': 'mg', 'Modele': '4', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Electrique', 'Puissan

moch mawjouda
{'Marque': 'audi', 'Modele': 'q2', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '8CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2023/10/47018.png?t=1717766921', 'Prix': '159 000 DT'}
moch mawjouda
{'Marque': 'audi', 'Modele': 'q3', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '9CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/06/47151.png?t=1717767845', 'Prix': '205 000 DT'}
moch mawjouda
{'Marque': 'audi', 'Modele': 'q3-sportback', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '9CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/07/47185.jpg?t=1721982793', 'Prix': '235 000 DT'}
moch mawjouda
{'Marque': 'audi', 'Modele': 'q5-sportback', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '16C

moch mawjouda
{'Marque': 'bmw', 'Modele': 'ix', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Electrique', 'Puissance fiscale': '11CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/05/47132.jpg?t=1720857841', 'Prix': '431 900 DT'}
moch mawjouda
{'Marque': 'byd', 'Modele': 'dolphin', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Electrique', 'Puissance fiscale': '5CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/06/47172.jpg?t=1723020945', 'Prix': '94 990 DT'}
moch mawjouda
{'Marque': 'byd', 'Modele': 'king', 'Nombre de places': '5', 'Nombre de portes': '4', 'Energie': 'Essence | Hybride rechargeable', 'Puissance fiscale': '5CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/06/47173.jpg?t=1721140422', 'Prix': '105 990 DT'}
moch mawjouda
{'Marque': 'byd', 'Modele': 'atto-3', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Electrique', 'Puiss

moch mawjouda
{'Marque': 'dfsk', 'Modele': 'glory-500', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '7CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2023/12/47055.jpg?t=1717152819', 'Prix': '83 900 DT'}
moch mawjouda
{'Marque': 'dfsk', 'Modele': 'glory-580', 'Nombre de places': '7', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '8CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2022/03/46718.jpg?t=1713019608', 'Prix': '99 900 DT'}
moch mawjouda
{'Marque': 'dfsk', 'Modele': 'glory-ix5', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '9CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/08/47202.jpg?t=1724404958', 'Prix': '118 900 DT'}
ida ken mawjouda comparer
{'Marque': 'dfsk', 'Modele': 'k01s', 'Nombre de places': '2', 'Nombre de portes': '2', 'Energie': 'Essence', 'Puissance f

moch mawjouda
{'Marque': 'gac-motor', 'Modele': 'gs3', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '7CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/03/46779.jpg?t=1721059621', 'Prix': '79 970 DT'}
moch mawjouda
{'Marque': 'geely', 'Modele': 'gx3-pro', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '6CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/02/47094.jpg?t=1725617176', 'Prix': '68 800 DT'}
ida ken mawjouda comparer
{'Marque': 'geely', 'Modele': 'coolray', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '10CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2022/10/46842.jpg?t=1725632424', 'Prix': '86 800 DT'}
moch mawjouda
{'Marque': 'geely', 'Modele': 'azkara', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence | Hybride lége

...

moch mawjouda
{'Marque': 'mg', 'Modele': 'marvel-r', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Electrique', 'Puissance fiscale': '5CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/08/47187.jpg?t=1723278434', 'Prix': '149 950 DT'}
moch mawjouda
{'Marque': 'mg', 'Modele': 'rx8', 'Nombre de places': '7', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '16CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2022/05/46748.jpg?t=1707860686', 'Prix': '174 950 DT'}
moch mawjouda
{'Marque': 'mini', 'Modele': '3-portes', 'Nombre de places': '5', 'Nombre de portes': '3', 'Energie': 'Essence', 'Puissance fiscale': '9CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/09/47226.jpg?t=1727264933', 'Prix': '198 900 DT'}
ida ken mawjouda comparer
{'Marque': 'mini', 'Modele': 'countryman', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance f

moch mawjouda
{'Marque': 'peugeot', 'Modele': 'expert-combi', 'Nombre de places': '9', 'Nombre de portes': '5 | 1 porte latérale coulissante', 'Energie': 'Diesel', 'Puissance fiscale': '8CV', 'Boîte': 'Manuelle', 'Image URL': 'https://catalogue.automobile.tn/big/2020/03/46358.jpg?t=1704281520', 'Prix': '151 900 DT'}
moch mawjouda
{'Marque': 'peugeot', 'Modele': 'traveller', 'Nombre de places': '9', 'Nombre de portes': '5 | 1 porte latérale coulissante', 'Energie': 'Diesel', 'Puissance fiscale': '8CV', 'Boîte': 'Manuelle', 'Image URL': 'https://catalogue.automobile.tn/big/2020/03/46343.jpg?t=1704281538', 'Prix': '171 900 DT'}
ida ken mawjouda comparer
{'Marque': 'peugeot', 'Modele': 'partner', 'Nombre de places': '2', 'Nombre de portes': '3 | 1 porte latérale coulissante | 2 portes arrières battantes tolées', 'Energie': 'Diesel', 'Puissance fiscale': '5CV', 'Boîte': 'Manuelle', 'Image URL': 'https://catalogue.automobile.tn/big/2024/06/47156.png?t=1718285901', 'Prix': '67 490 DT'}
moch m

ida ken mawjouda comparer
{'Marque': 'skoda', 'Modele': 'scala', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '6CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/05/47146.jpg?t=1716214751', 'Prix': '88 980 DT'}
ida ken mawjouda comparer
{'Marque': 'skoda', 'Modele': 'kamiq', 'Nombre de places': '5', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '6CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/02/47103.png?t=1707325779', 'Prix': '91 980 DT'}
moch mawjouda
{'Marque': 'skoda', 'Modele': 'octavia-rs', 'Nombre de places': '5', 'Nombre de portes': '4', 'Energie': 'Essence', 'Puissance fiscale': '14CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2021/10/46662.jpg?t=1704365679', 'Prix': '216 980 DT'}
moch mawjouda
{'Marque': 'skoda', 'Modele': 'superb', 'Nombre de places': '5', 'Nombre de portes': '4', 'Energie': 'Essence', 

moch mawjouda
{'Marque': 'toyota', 'Modele': 'land-cruiser-76', 'Nombre de places': '9', 'Nombre de portes': '4 | 2 portes arrières battantes vitrées', 'Energie': 'Diesel', 'Puissance fiscale': '11CV', 'Boîte': 'Manuelle', 'Image URL': 'https://catalogue.automobile.tn/big/2021/11/46593.jpg?t=1655133474', 'Prix': '224 998 DT'}
moch mawjouda
{'Marque': 'toyota', 'Modele': 'fortuner', 'Nombre de places': '7', 'Nombre de portes': '5', 'Energie': 'Diesel', 'Puissance fiscale': '8CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/05/47130.jpg?t=1716559704', 'Prix': '274 800 DT'}
ida ken mawjouda comparer
{'Marque': 'toyota', 'Modele': 'prado', 'Nombre de places': '7', 'Nombre de portes': '5', 'Energie': 'Diesel', 'Puissance fiscale': '11CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2024/07/47180.jpg?t=1721727552', 'Prix': '342 000 DT'}
ida ken mawjouda comparer
{'Marque': 'toyota', 'Modele': 'land-cruiser-300', 'Nombre de plac

ida ken mawjouda comparer
{'Marque': 'volvo', 'Modele': 'xc90', 'Nombre de places': '7', 'Nombre de portes': '5', 'Energie': 'Essence | Hybride léger', 'Puissance fiscale': '15CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2023/08/46977.jpg?t=1724159911', 'Prix': '375 900 DT'}
moch mawjouda
{'Marque': 'wallyscar', 'Modele': 'wolf', 'Nombre de places': '7', 'Nombre de portes': '5', 'Energie': 'Essence', 'Puissance fiscale': '9CV', 'Boîte': 'Automatique', 'Image URL': 'https://catalogue.automobile.tn/big/2023/12/47039.png?t=1723647188', 'Prix': '119 900 DT'}


In [22]:
df

,Marque,Modele,Nombre de places,Nombre de portes,Energie,Puissance fiscale,Boîte,Image URL,Prix
0,bako,bee,3,2,Electrique,1CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,16 210 DT
1,bako,evan,1,3,Electrique,N.C,Automatique,https://catalogue.automobile.tn/big/2024/08/47...,22 586 DT
2,byd,dolphin,5,5,Electrique,5CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,94 990 DT
3,mg,4,5,5,Electrique,5CV,NaN,https://catalogue.automobile.tn/big/2024/08/47...,99 950 DT
4,mg,ezs,5,5,Electrique,9CV,NaN,https://catalogue.automobile.tn/big/2024/02/47...,116 950 DT
...,...,...,...,...,...,...,...,...,...
306,volvo,c40-recharge,5,5,Electrique,9CV,Automatique,https://catalogue.automobile.tn/big/2024/02/47...,199 900 DT
307,volvo,xc60,5,5,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2023/05/46...,289 900 DT
308,volvo,s90,5,4,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2024/09/47...,259 900 DT
309,volvo,xc90,7,5,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2023/08/46...,375 900 DT


In [23]:
# SAVE
df.to_csv("all_cars11.csv", index=False)

# MERG ALL DATA 

In [50]:
import pandas as pd
# tansech lpost mta3 nada
mercedes = pd.read_csv("mercedes-benz.csv")
audi = pd.read_csv("audi.csv")
BMW = pd.read_csv("BMW.csv")
fiat = pd.read_csv("fiat.csv")
ford = pd.read_csv("ford.csv")

In [51]:
df = pd.concat([audi, BMW, fiat, ford, mercedes], axis=0)
df = df.reset_index(drop=True)
df

,Marque,Modele,Nombre de places,Nombre de portes,Energie,Puissance fiscale,Boîte,Image URL,Prix
0,audi,a3-sportback,5,5,Essence,8CV,Automatique,https://catalogue.automobile.tn/big/2024/02/46...,154 000 DT
1,audi,a3-berline,5,4,Essence,8CV,Automatique,https://catalogue.automobile.tn/big/2024/02/47...,156 000 DT
2,audi,a4,5,4,Essence,8CV,Automatique,https://catalogue.automobile.tn/big/2024/01/47...,198 000 DT
3,audi,a6,5,4,Essence | Hybride léger,11CV,Automatique,https://catalogue.automobile.tn/big/2023/08/46...,295 000 DT
4,audi,q2,5,5,Essence,8CV,Automatique,https://catalogue.automobile.tn/big/2023/10/47...,159 000 DT
5,audi,q3,5,5,Essence,9CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,205 000 DT
6,audi,q3-sportback,5,5,Essence,9CV,Automatique,https://catalogue.automobile.tn/big/2024/07/47...,235 000 DT
7,audi,q5-sportback,5,5,Essence,16CV,Automatique,https://catalogue.automobile.tn/big/2022/06/46...,350 000 DT
8,audi,q7,7,5,Essence,16CV,Automatique,https://catalogue.automobile.tn/big/2022/10/46...,445 000 DT
9,audi,q8,5,5,Essence | Hybride léger,23CV,Automatique,https://catalogue.automobile.tn/big/2022/10/45...,510 000 DT


In [52]:
df.to_csv("all_cars.csv", index=False)

# Cleaning Data

In [24]:
df = pd.read_csv("all_cars11.csv")
df

,Marque,Modele,Nombre de places,Nombre de portes,Energie,Puissance fiscale,Boîte,Image URL,Prix
0,bako,bee,3,2,Electrique,1CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,16 210 DT
1,bako,evan,1,3,Electrique,N.C,Automatique,https://catalogue.automobile.tn/big/2024/08/47...,22 586 DT
2,byd,dolphin,5,5,Electrique,5CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,94 990 DT
3,mg,4,5,5,Electrique,5CV,NaN,https://catalogue.automobile.tn/big/2024/08/47...,99 950 DT
4,mg,ezs,5,5,Electrique,9CV,NaN,https://catalogue.automobile.tn/big/2024/02/47...,116 950 DT
...,...,...,...,...,...,...,...,...,...
306,volvo,c40-recharge,5,5,Electrique,9CV,Automatique,https://catalogue.automobile.tn/big/2024/02/47...,199 900 DT
307,volvo,xc60,5,5,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2023/05/46...,289 900 DT
308,volvo,s90,5,4,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2024/09/47...,259 900 DT
309,volvo,xc90,7,5,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2023/08/46...,375 900 DT


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Marque             311 non-null    object
 1   Modele             311 non-null    object
 2   Nombre de places   311 non-null    int64 
 3   Nombre de portes   311 non-null    object
 4   Energie            311 non-null    object
 5   Puissance fiscale  311 non-null    object
 6   Boîte              307 non-null    object
 7   Image URL          311 non-null    object
 8   Prix               311 non-null    object
dtypes: int64(1), object(8)
memory usage: 22.0+ KB


In [26]:
df["Marque"].unique()

array(['bako', 'byd', 'mg', 'hyundai', 'volvo', 'kia', 'bmw', 'audi',
       'porsche', 'alfa-romeo', 'bestune', 'changan', 'chery',
       'chevrolet', 'citroen', 'cupra', 'dacia', 'dfsk', 'dongfeng',
       'faw', 'fiat', 'foday', 'ford', 'gac-motor', 'geely', 'great-wall',
       'haval', 'honda', 'jaguar', 'jeep', 'land-rover', 'mahindra',
       'mercedes-benz', 'mini', 'mitsubishi', 'nissan', 'opel', 'peugeot',
       'renault', 'seat', 'skoda', 'ssangyong', 'suzuki', 'tata',
       'toyota', 'volkswagen', 'wallyscar'], dtype=object)

In [27]:
df["Modele"].unique()

array(['bee', 'evan', 'dolphin', '4', 'ezs', 'kona-ev', 'atto-3', 'ex30',
       'marvel-r', 'ioniq-5', 'c40-recharge', 'ev6', 'ix1', 'tang-ev',
       'i4', 'ev6-gt', 'ix3', 'q8-e-tron', 'q8-sportback-e-tron',
       'e-tron-gt', 'ix', 'taycan', 'taycan-cross-turismo', 'giulia',
       'stelvio', 'a3-sportback', 'a3-berline', 'a4', 'a6', 'q2', 'q3',
       'q3-sportback', 'q5-sportback', 'q7', 'q8', 'bestune-t77',
       'serie-1-5p', 'serie-2-gran-coupe', 'serie-3',
       'serie-4-gran-coupe', 'serie-4-coupe', 'serie-5', 'serie-7', 'x1',
       'x1-hybride', 'x2', 'x3-hybride-rechargeable', 'x4', 'x5-hybride',
       'king', 'song-plus', 'new-star', 'star-truck-double-cabine',
       'hunter', 'tiggo-1x-populaire', 'tiggo-3x', 'arrizo-5', 'tiggo-7',
       'tiggo-8-pro', 'sonic', 'groove', 'captiva', 'equinox',
       'c3-populaire', 'berlingo-van', 'jumpy-fourgon', 'jumper', 'leon',
       'formentor', 'sandero-stepway', 'sandero', 'logan', 'duster',
       'glory-500', 'glory-580'

In [28]:
df["Nombre de places"].unique()

array([ 3,  1,  5,  7,  2,  4, 15, 11,  9,  6, 16, 23], dtype=int64)

In [29]:
df["Nombre de places"] = df["Nombre de places"].astype(int)

In [30]:
df["Nombre de portes"].unique()

array(['2', '3', '5', '4', '3 | 2 portes latérales coulissantes',
       '5 | 1 porte latérale coulissante | 2 portes arrières battantes tolées',
       '5 | 2 portes latérales coulissantes',
       '2 portes latérales coulissantes | 2 portes arrières battantes tolées | 6',
       '5 | 1 porte latérale coulissante',
       '3 | 1 porte latérale coulissante | 2 portes arrières battantes tolées',
       '4 | 1 porte latérale coulissante | 2 portes arrières battantes tolées',
       '4 | 1 porte latérale coulissante | 2 portes arrières battantes vitrées',
       '2 | 1 porte latérale coulissante | 2 portes arrières battantes tolées',
       '4 | 2 portes arrières battantes vitrées',
       '2 | 1 porte latérale battante',
       '3 | 1 porte latérale coulissante',
       '4 | 1 porte latérale coulissante'], dtype=object)

In [31]:
import re 
def extract_number(text):
    numbers = re.findall(r'\d+', text)
    numbers = list(map(int, numbers))

    if not numbers:
        return None
    else:
        return numbers[0] 
    
df['Nombre de portes'] = df['Nombre de portes'].apply(lambda x: extract_number(x))
df['Nombre de portes'] = df['Nombre de portes'].astype(int)
df

,Marque,Modele,Nombre de places,Nombre de portes,Energie,Puissance fiscale,Boîte,Image URL,Prix
0,bako,bee,3,2,Electrique,1CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,16 210 DT
1,bako,evan,1,3,Electrique,N.C,Automatique,https://catalogue.automobile.tn/big/2024/08/47...,22 586 DT
2,byd,dolphin,5,5,Electrique,5CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,94 990 DT
3,mg,4,5,5,Electrique,5CV,NaN,https://catalogue.automobile.tn/big/2024/08/47...,99 950 DT
4,mg,ezs,5,5,Electrique,9CV,NaN,https://catalogue.automobile.tn/big/2024/02/47...,116 950 DT
...,...,...,...,...,...,...,...,...,...
306,volvo,c40-recharge,5,5,Electrique,9CV,Automatique,https://catalogue.automobile.tn/big/2024/02/47...,199 900 DT
307,volvo,xc60,5,5,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2023/05/46...,289 900 DT
308,volvo,s90,5,4,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2024/09/47...,259 900 DT
309,volvo,xc90,7,5,Essence | Hybride léger,15CV,Automatique,https://catalogue.automobile.tn/big/2023/08/46...,375 900 DT


In [32]:
df["Energie"].unique()

array(['Electrique', 'Essence', 'Essence | Hybride léger',
       'Essence | Hybride rechargeable', 'Diesel', 'Essence | Hybride',
       'Diesel | Hybride léger'], dtype=object)

In [33]:
def extract_first_part(text):
    return text.split('|')[0].strip()

df['Energie'] = df['Energie'].apply(extract_first_part)
df

,Marque,Modele,Nombre de places,Nombre de portes,Energie,Puissance fiscale,Boîte,Image URL,Prix
0,bako,bee,3,2,Electrique,1CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,16 210 DT
1,bako,evan,1,3,Electrique,N.C,Automatique,https://catalogue.automobile.tn/big/2024/08/47...,22 586 DT
2,byd,dolphin,5,5,Electrique,5CV,Automatique,https://catalogue.automobile.tn/big/2024/06/47...,94 990 DT
3,mg,4,5,5,Electrique,5CV,NaN,https://catalogue.automobile.tn/big/2024/08/47...,99 950 DT
4,mg,ezs,5,5,Electrique,9CV,NaN,https://catalogue.automobile.tn/big/2024/02/47...,116 950 DT
...,...,...,...,...,...,...,...,...,...
306,volvo,c40-recharge,5,5,Electrique,9CV,Automatique,https://catalogue.automobile.tn/big/2024/02/47...,199 900 DT
307,volvo,xc60,5,5,Essence,15CV,Automatique,https://catalogue.automobile.tn/big/2023/05/46...,289 900 DT
308,volvo,s90,5,4,Essence,15CV,Automatique,https://catalogue.automobile.tn/big/2024/09/47...,259 900 DT
309,volvo,xc90,7,5,Essence,15CV,Automatique,https://catalogue.automobile.tn/big/2023/08/46...,375 900 DT


In [34]:
# df['Puissance fiscale'] = df['Puissance fiscale'].str.replace('CV', '').str.strip()
# df['Puissance fiscale'] = df['Puissance fiscale'].astype(int)
# df

In [35]:
# df['Prix'] = df['Prix'].str.replace('DT', '').str.strip()
# df['Prix'] = df['Prix'].str.replace(' ', '').str.strip()
# df['Prix'] = df['Prix'].astype(int)
# df

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311 entries, 0 to 310
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Marque             311 non-null    object
 1   Modele             311 non-null    object
 2   Nombre de places   311 non-null    int32 
 3   Nombre de portes   311 non-null    int32 
 4   Energie            311 non-null    object
 5   Puissance fiscale  311 non-null    object
 6   Boîte              307 non-null    object
 7   Image URL          311 non-null    object
 8   Prix               311 non-null    object
dtypes: int32(2), object(7)
memory usage: 19.6+ KB


In [37]:
df.to_csv("all_cars22.csv", index=False)